In [1]:
import os
import pandas as pd
import numpy as np
from scipy.sparse import csr_matrix
from sklearn.model_selection import train_test_split
from scipy.sparse.linalg import svds
from sklearn.metrics import mean_squared_error, precision_score, recall_score, f1_score
from sklearn.decomposition import TruncatedSVD
from sklearn.pipeline import make_pipeline
from pyspark.sql import SparkSession
from pyspark.ml.recommendation import ALS
from pyspark.ml.evaluation import RegressionEvaluator
from pyspark.ml.tuning import CrossValidator, ParamGridBuilder

import tensorflow as tf
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Embedding, Flatten, Concatenate, Dense, Dropout
from tensorflow.keras.regularizers import l2
from tensorflow.keras.optimizers import Adam
from itertools import product

### Data Loading
We start by loading the preprocessed user-item matrix saved from the data cleaning step. This data will be split into training and testing sets for model development.

In [2]:
# Read the CSV file into a DataFrame
#Load the user-item matrix
user_item = os.path.join("..", "Data", "Cleaned-Data", 'user_item_filtered.csv')
user_item_df = pd.read_csv(user_item)

# Convert the DataFrame to a sparse matrix
user_item_filtered = csr_matrix(user_item_df.values)

### Train/Test Split
We split the user-item interaction matrix into training and testing subsets. This allows us to train our models on a portion of the data and evaluate their performance on unseen data.

In [3]:
# Split the user-item interaction matrix into train and test subsets
train_data, test_data = train_test_split(user_item_filtered, test_size=0.2, random_state=42)

### Model 1: Singular Value Decomposition (SVD)
The first model we develop is based on SVD. SVD is a popular technique for matrix factorization in recommendation systems, especially useful for capturing latent features in sparse matrices.

In [4]:
# Perform SVD on the training data
u, s, vt = svds(train_data, k=100)

# Convert singular values to a diagonal matrix
s_diag = np.diag(s)

# Predict ratings for the test data
predicted_ratings = np.dot(np.dot(u, s_diag), vt)

# Get the non-zero indices of the test data
test_nonzero_indices = test_data.nonzero()

# Get the actual ratings from the test data
actual_ratings = np.array(test_data[test_nonzero_indices].data).ravel()

# Get the predicted ratings for the non-zero indices
predicted_ratings_nonzero = predicted_ratings[test_nonzero_indices[0], test_nonzero_indices[1]]

# Ensure predicted_ratings_nonzero is a 1D array
predicted_ratings_nonzero = predicted_ratings_nonzero.ravel()

# Debug: print lengths to check consistency
print("Length of actual ratings:", actual_ratings.shape)
print("Length of predicted ratings:", predicted_ratings_nonzero.shape)

# Evaluate the model using RMSE
mse = mean_squared_error(actual_ratings, predicted_ratings_nonzero)
rmse = np.sqrt(mse)
print("RMSE:", rmse)

Length of actual ratings: (1824,)
Length of predicted ratings: (1824,)
RMSE: 0.9996157395847921


That was a really bad model considering the RMSE is so close to 1. Maybe some other metrics would help us here? 

In [5]:
# Compute precision, recall, and F1-score for SVD
precision_svd = precision_score(actual_ratings, predicted_ratings_nonzero.round())
recall_svd = recall_score(actual_ratings, predicted_ratings_nonzero.round())
f1_svd = f1_score(actual_ratings, predicted_ratings_nonzero.round())

print("SVD Evaluation:")
print("Precision:", precision_svd)
print("Recall:", recall_svd)
print("F1-score:", f1_svd)

SVD Evaluation:
Precision: 0.0
Recall: 0.0
F1-score: 0.0


/Users/jin/anaconda3/lib/python3.11/site-packages/sklearn/metrics/_classification.py:1469: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [6]:
# Create a Truncated SVD instance
svd = TruncatedSVD(n_components=100, random_state=42)

In [7]:
# Perform SVD on the training data
lsa = make_pipeline(svd)
train_data_reduced = lsa.fit_transform(train_data)

# Transform the test data using the same model
test_data_reduced = lsa.transform(test_data)

# Compute predictions (for binary you might need to threshold these)
predicted_ratings_l2 = np.dot(train_data_reduced, svd.components_)

In [8]:
# Get the non-zero indices of the test data
test_nonzero_indices = test_data.nonzero()

# Extract actual ratings for the non-zero indices
actual_ratings = np.array(test_data[test_nonzero_indices]).ravel()

# Extract predicted ratings for the non-zero indices
predicted_ratings_nonzero_l2 = predicted_ratings_l2[test_nonzero_indices].ravel()

# Evaluate the model using RMSE
mse = mean_squared_error(actual_ratings, predicted_ratings_nonzero_l2)
rmse = np.sqrt(mse)
print("RMSE:", rmse)

RMSE: 0.999647555897552


That was a really bad model considering the RMSE is so close to 1. Maybe some other metrics would help us here? (Same explanaton as the other one.) 

In [9]:
# Compute precision, recall, and F1-score for Truncated SVD
precision_tsvd = precision_score(actual_ratings, predicted_ratings_nonzero_l2.round())
recall_tsvd = recall_score(actual_ratings, predicted_ratings_nonzero_l2.round())
f1_tsvd = f1_score(actual_ratings, predicted_ratings_nonzero_l2.round())


print("\nTruncated SVD Evaluation:")
print("Precision:", precision_tsvd)
print("Recall:", recall_tsvd)
print("F1-score:", f1_tsvd)


Truncated SVD Evaluation:
Precision: 0.0
Recall: 0.0
F1-score: 0.0


/Users/jin/anaconda3/lib/python3.11/site-packages/sklearn/metrics/_classification.py:1469: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


It seems as if both the SVD and TSVD are encountering a divide by zero error, this indicates that both models are not predicting any positive interactions correctly. This might be due to the sparsity of the user-item matrix. 

### Model 2: Alternating Least Squares (ALS)
The next model we explore is ALS. ALS is designed to handle implicit feedback by assigning confidence levels to interactions, making it well-suited for large-scale recommendation systems.

In [34]:
# Create a Spark session
spark = SparkSession.builder.appName("CollaborativeFiltering").config("spark.executor.memory", "8g").config("spark.driver.memory", "8g").config("spark.sql.execution.arrow.pyspark.enabled", "true").getOrCreate()

In [35]:
# Suppress log warnings
spark.sparkContext.setLogLevel("ERROR")

In [36]:
# Convert the user-item interaction matrix to a Spark DataFrame
user_item_df_spark = spark.createDataFrame(
    [(i, j, float(user_item_df.iloc[i, j])) for i in range(user_item_df.shape[0]) for j in range(user_item_df.shape[1])],
    ["user", "item", "rating"]
)

#Needed to reduce the size of the matrix to account for hardware constraints on my machine
user_item_df_spark_sample = user_item_df_spark.sample(fraction=0.1, seed=42)

In [37]:
# Split the data into training and testing sets
#(training_data_spark, test_data_spark) = user_item_df_spark.randomSplit([0.8, 0.2])

(training_data_spark, test_data_spark) = user_item_df_spark_sample.randomSplit([0.8, 0.2])

In [38]:
# Create an ALS model
als = ALS(maxIter=10, regParam=0.01, userCol="user", itemCol="item", ratingCol="rating",
          coldStartStrategy="drop")

In [39]:
# Train the model
model = als.fit(training_data_spark)

In [40]:
# Generate predictions on the test data
predictions = model.transform(test_data_spark)

In [41]:
# Evaluate the model
evaluator = RegressionEvaluator(metricName="rmse", labelCol="rating", predictionCol="prediction")
rmse = evaluator.evaluate(predictions)
print("Root-mean-square error = " + str(rmse))

Root-mean-square error = 0.020309390684188168


In [42]:
# Generate top-k recommendations for each user
user_recs = model.recommendForAllUsers(10)
user_recs.show(truncate=False)

+----+-----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+
|user|recommendations                                                                                                                                                                                                                    |
+----+-----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+
|12  |[{3403, 8.107912E-21}, {288, 3.209575E-21}, {3099, 3.061551E-21}, {2479, 1.6572113E-21}, {4104, 1.6190934E-21}, {387, 1.3059998E-21}, {3287, 1.2951236E-21}, {534, 1.1945253E-21}, {3532, 9.683101E-22}, {3698, 8.18446E-22}]      |
|22  |[{1176, 1.9878367E-20}, {3403, 6.0629556E-21}, {4104, 

In [43]:
# Build the parameter grid
param_grid = (ParamGridBuilder().addGrid(als.rank, [10, 20]).addGrid(als.regParam, [0.01, 1.0]).addGrid(als.maxIter, [10]).build())

In [44]:
# Create a cross-validator
cross_validator = CrossValidator(estimator=als, estimatorParamMaps=param_grid, evaluator=evaluator,numFolds=5)

In [46]:
# Run cross-validation to find the best model
cv_model = cross_validator.fit(training_data_spark)

In [47]:
# Get the best model from cross-validation
best_model = cv_model.bestModel

In [48]:
# Generate predictions on the test data
best_predictions = best_model.transform(test_data_spark)

In [49]:
# Evaluate the best model
best_rmse = evaluator.evaluate(best_predictions)
print("Best RMSE: ", best_rmse)

Best RMSE:  0.020309390684188168


In [50]:
# Generate top-k recommendations for each user using the best model
best_user_recs = best_model.recommendForAllUsers(10)
best_user_recs.show(truncate=False)

+----+-----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+
|user|recommendations                                                                                                                                                                                                                    |
+----+-----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+
|12  |[{3403, 8.107912E-21}, {288, 3.209575E-21}, {3099, 3.061551E-21}, {2479, 1.6572113E-21}, {4104, 1.6190934E-21}, {387, 1.3059998E-21}, {3287, 1.2951236E-21}, {534, 1.1945253E-21}, {3532, 9.683101E-22}, {3698, 8.18446E-22}]      |
|22  |[{1176, 1.9878367E-20}, {3403, 6.0629556E-21}, {4104, 

In [51]:
# Convert predictions to Pandas DataFrame for easier evaluation
#predictions_df = best_predictions.toPandas()

# Limit the number of rows for testing purposes
limited_predictions = best_predictions.limit(500)
predictions_df = limited_predictions.toPandas()

In [52]:
# Round predictions to get binary values
predictions_df['prediction'] = predictions_df['prediction'].round().astype(int)

In [53]:
# Compute precision, recall, and F1-score
precision = precision_score(predictions_df['rating'], predictions_df['prediction'], zero_division=1)
recall = recall_score(predictions_df['rating'], predictions_df['prediction'], zero_division=1)
f1 = f1_score(predictions_df['rating'], predictions_df['prediction'], zero_division=1)

print("Precision: ", precision)
print("Recall: ", recall)
print("F1-Score: ", f1)

Precision:  1.0
Recall:  1.0
F1-Score:  1.0


In [58]:
# Retrieve the metrics for each parameter combination
results = []
param_maps = cv_model.getEstimatorParamMaps()
for i, avg_metric in enumerate(cv_model.avgMetrics):
    results.append((avg_metric, param_maps[i]))

# Sort the results by the evaluation metric
results.sort(key=lambda x: x[0])

# Evaluate the top 4 parameter combinations
for i, (metric, params) in enumerate(results[:4]):
    print(f"Top {i+1} Model - Metric: {metric}, Params: {params}")
    model = als.fit(training_data_spark)
    predictions = model.transform(test_data_spark)
    
    # Compute precision, recall, and F1-score
    precision = precision_score(predictions_df['rating'], predictions_df['prediction'], zero_division=1)
    recall = recall_score(predictions_df['rating'], predictions_df['prediction'], zero_division=1)
    f1 = f1_score(predictions_df['rating'], predictions_df['prediction'], zero_division=1)
    
    print(f"Top {i+1} Model Evaluation:")
    print(f"Precision: {precision}")
    print(f"Recall: {recall}")
    print(f"F1-Score: {f1}")

Top 1 Model - Metric: 0.02005747826643014, Params: {Param(parent='ALS_f2a50bb1b2bd', name='rank', doc='rank of the factorization'): 10, Param(parent='ALS_f2a50bb1b2bd', name='regParam', doc='regularization parameter (>= 0).'): 0.01, Param(parent='ALS_f2a50bb1b2bd', name='maxIter', doc='max number of iterations (>= 0).'): 10}


Top 1 Model Evaluation:
Precision: 1.0
Recall: 1.0
F1-Score: 1.0
Top 2 Model - Metric: 0.02005747826643014, Params: {Param(parent='ALS_f2a50bb1b2bd', name='rank', doc='rank of the factorization'): 10, Param(parent='ALS_f2a50bb1b2bd', name='regParam', doc='regularization parameter (>= 0).'): 1.0, Param(parent='ALS_f2a50bb1b2bd', name='maxIter', doc='max number of iterations (>= 0).'): 10}


Top 2 Model Evaluation:
Precision: 1.0
Recall: 1.0
F1-Score: 1.0
Top 3 Model - Metric: 0.02005747826643014, Params: {Param(parent='ALS_f2a50bb1b2bd', name='rank', doc='rank of the factorization'): 20, Param(parent='ALS_f2a50bb1b2bd', name='regParam', doc='regularization parameter (>= 0).'): 0.01, Param(parent='ALS_f2a50bb1b2bd', name='maxIter', doc='max number of iterations (>= 0).'): 10}


Top 3 Model Evaluation:
Precision: 1.0
Recall: 1.0
F1-Score: 1.0
Top 4 Model - Metric: 0.02005747826643014, Params: {Param(parent='ALS_f2a50bb1b2bd', name='rank', doc='rank of the factorization'): 20, Param(parent='ALS_f2a50bb1b2bd', name='regParam', doc='regularization parameter (>= 0).'): 1.0, Param(parent='ALS_f2a50bb1b2bd', name='maxIter', doc='max number of iterations (>= 0).'): 10}


Top 4 Model Evaluation:
Precision: 1.0
Recall: 1.0
F1-Score: 1.0


In [59]:
# Stop the Spark session
spark.stop()

### Model 3: Deep Learning
The final model we implement is a deep learning model using TensorFlow. Deep learning models can capture complex, non-linear relationships in the data, providing a flexible approach to learning user preferences.

In [60]:
#Attempting with deep learning
# Prepare the data for the deep learning model
train_data_nonzero = train_data.nonzero()
test_data_nonzero = test_data.nonzero()

train_user = train_data_nonzero[0]
train_item = train_data_nonzero[1]
train_rating = train_data[train_user, train_item].A1

test_user = test_data_nonzero[0]
test_item = test_data_nonzero[1]
test_rating = test_data[test_user, test_item].A1

In [61]:
# Define the deep learning model
num_users = user_item_df.shape[0]
num_items = user_item_df.shape[1]

user_input = Input(shape=(1,))
item_input = Input(shape=(1,))

user_embedding = Embedding(num_users, 50)(user_input)
item_embedding = Embedding(num_items, 50)(item_input)

user_vector = Flatten()(user_embedding)
item_vector = Flatten()(item_embedding)

concat = Concatenate()([user_vector, item_vector])

dense = Dense(128, activation='relu', kernel_regularizer=l2(0.01))(concat)
dropout = Dropout(0.5)(dense)
output = Dense(1)(dropout)

model = Model(inputs=[user_input, item_input], outputs=output)
model.compile(optimizer='adam', loss='mean_squared_error')

In [62]:
# Implement early stopping
early_stopping = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=3)

In [63]:
# Train the model
model.fit([train_user, train_item], train_rating, epochs=50, batch_size=64, validation_split=0.1, callbacks=[early_stopping])

Epoch 1/50
103/103 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - loss: 1.3532 - val_loss: 0.1400
Epoch 2/50
103/103 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 0.1005 - val_loss: 0.0095
Epoch 3/50
103/103 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 0.0194 - val_loss: 5.4236e-04
Epoch 4/50
103/103 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 0.0143 - val_loss: 2.9510e-04
Epoch 5/50
103/103 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 0.0131 - val_loss: 2.7287e-04
Epoch 6/50
103/103 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 0.0131 - val_loss: 2.6825e-04
Epoch 7/50
103/103 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 0.0130 - val_loss: 4.6576e-04
Epoch 8/50
103/103 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 0.0127 - val_loss: 2.4031e-04
Epoch 9/50
103/103 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 0.0130 - val_loss: 3.8630e-04
Epoch 10/50
103/103 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 0.0125 - val_loss: 3.8269e-04
Epoch 11/50
103/103 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 0.0116 - val_loss: 4.3646e-04


In [64]:
# Predict ratings for the test data
predicted_ratings = model.predict([test_user, test_item])

57/57 ━━━━━━━━━━━━━━━━━━━━ 0s 369us/step


In [65]:
# Evaluate the model using RMSE
rmse = np.sqrt(mean_squared_error(test_rating, predicted_ratings))
print("Deep Learning Model RMSE:", rmse)

Deep Learning Model RMSE: 0.015109668227329965


In [66]:
# Compute precision, recall, and F1-score
predicted_ratings_rounded = np.round(predicted_ratings).astype(int)

precision_dl = precision_score(test_rating, predicted_ratings_rounded, zero_division=1)
recall_dl = recall_score(test_rating, predicted_ratings_rounded, zero_division=1)
f1_dl = f1_score(test_rating, predicted_ratings_rounded, zero_division=1)

print("Deep Learning Model Evaluation:")
print("Precision:", precision_dl)
print("Recall:", recall_dl)
print("F1-score:", f1_dl)

Deep Learning Model Evaluation:
Precision: 1.0
Recall: 1.0
F1-score: 1.0


### Hyperparameter tuning for Deep Learning

In [67]:
# Define the hyperparameter grid
param_grid = {
    'embedding_dim': [20, 50],
    'dense_units': [64, 128],
    'dropout_rate': [0.3, 0.5],
    'learning_rate': [1e-3, 1e-2],
    'l2_reg': [0.01, 0.05]
}

In [68]:
# Create a list of all hyperparameter combinations
all_params = list(product(param_grid['embedding_dim'], param_grid['dense_units'], param_grid['dropout_rate'], param_grid['learning_rate'], param_grid['l2_reg']))

In [69]:
# Initialize a list to store the results
results = []

In [70]:
# Iterate through each combination of hyperparameters
for params in all_params:
    embedding_dim, dense_units, dropout_rate, learning_rate, l2_reg = params
    
    # Define the deep learning model
    user_input = Input(shape=(1,))
    item_input = Input(shape=(1,))

    user_embedding = Embedding(num_users, embedding_dim)(user_input)
    item_embedding = Embedding(num_items, embedding_dim)(item_input)

    user_vector = Flatten()(user_embedding)
    item_vector = Flatten()(item_embedding)

    concat = Concatenate()([user_vector, item_vector])

    dense = Dense(dense_units, activation='relu', kernel_regularizer=l2(l2_reg))(concat)
    dropout = Dropout(dropout_rate)(dense)
    output = Dense(1)(dropout)

    model = Model(inputs=[user_input, item_input], outputs=output)
    model.compile(optimizer=Adam(learning_rate), loss='mean_squared_error')

    # Implement early stopping
    early_stopping = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=3)

    # Train the model
    history = model.fit([train_user, train_item], train_rating, epochs=50, batch_size=64, validation_split=0.1, callbacks=[early_stopping], verbose=0)

    # Predict ratings for the test data
    predicted_ratings = model.predict([test_user, test_item])

    # Evaluate the model using RMSE
    rmse = np.sqrt(mean_squared_error(test_rating, predicted_ratings))

    # Compute precision, recall, and F1-score
    predicted_ratings_rounded = np.round(predicted_ratings).astype(int)

    precision_dl = precision_score(test_rating, predicted_ratings_rounded, zero_division=1)
    recall_dl = recall_score(test_rating, predicted_ratings_rounded, zero_division=1)
    f1_dl = f1_score(test_rating, predicted_ratings_rounded, zero_division=1)

    # Store the results
    results.append({
        'embedding_dim': embedding_dim,
        'dense_units': dense_units,
        'dropout_rate': dropout_rate,
        'learning_rate': learning_rate,
        'l2_reg': l2_reg,
        'rmse': rmse,
        'precision': precision_dl,
        'recall': recall_dl,
        'f1_score': f1_dl
    })

    print(f"Evaluated parameters: {params}")
    print(f"Results - RMSE: {rmse}, Precision: {precision_dl}, Recall: {recall_dl}, F1-score: {f1_dl}")

# Save the results to a DataFrame
results_df = pd.DataFrame(results)
results_df.to_csv('grid_search_results.csv', index=False)

# Print the best hyperparameters
best_params = results_df.loc[results_df['rmse'].idxmin()]
print("Best Hyperparameters:")
print(best_params)

57/57 ━━━━━━━━━━━━━━━━━━━━ 0s 312us/step
Evaluated parameters: (20, 64, 0.3, 0.001, 0.01)
Results - RMSE: 0.007902589648087335, Precision: 1.0, Recall: 1.0, F1-score: 1.0
57/57 ━━━━━━━━━━━━━━━━━━━━ 0s 274us/step
Evaluated parameters: (20, 64, 0.3, 0.001, 0.05)
Results - RMSE: 0.009924226905032437, Precision: 1.0, Recall: 1.0, F1-score: 1.0
57/57 ━━━━━━━━━━━━━━━━━━━━ 0s 294us/step
Evaluated parameters: (20, 64, 0.3, 0.01, 0.01)
Results - RMSE: 0.02525882580437974, Precision: 1.0, Recall: 1.0, F1-score: 1.0
57/57 ━━━━━━━━━━━━━━━━━━━━ 0s 282us/step
Evaluated parameters: (20, 64, 0.3, 0.01, 0.05)
Results - RMSE: 0.009214923362180808, Precision: 1.0, Recall: 1.0, F1-score: 1.0
57/57 ━━━━━━━━━━━━━━━━━━━━ 0s 282us/step
Evaluated parameters: (20, 64, 0.5, 0.001, 0.01)
Results - RMSE: 0.02524435276419191, Precision: 1.0, Recall: 1.0, F1-score: 1.0
57/57 ━━━━━━━━━━━━━━━━━━━━ 0s 288us/step
Evaluated parameters: (20, 64, 0.5, 0.001, 0.05)
Results - RMSE: 0.03773747028915559, Precision: 1.0, Recall

### Conclusion
We summarize the performance of each model, highlight their strengths and weaknesses, and outline the next steps for improving our recommendation system.

Summary of Model Evaluations

In this project, we developed and evaluated four different models to predict user preferences for books, music, and movies. Each model’s performance was assessed using various metrics, and the results are summarized below.

1. Singular Value Decomposition (SVD)

	•	Length of Actual Ratings: 1824
    
	•	Length of Predicted Ratings: 1824
    
	•	Root Mean Squared Error (RMSE): 0.9996
    
	•	Precision: 0.0
    
	•	Recall: 0.0
    
	•	F1-Score: 0.0
    

Analysis:
The SVD model achieved an RMSE of approximately 0.9996, indicating poor performance in predicting user ratings. The precision, recall, and F1-score were all 0.0, suggesting that the model failed to correctly identify relevant items. This poor performance could be due to the model’s assumption of linearity in latent factors, which may not hold true in our data.

2. Truncated SVD

	•	Root Mean Squared Error (RMSE): 0.9996
    
	•	Precision: 0.0
    
	•	Recall: 0.0
    
	•	F1-Score: 0.0
    

Analysis:
The Truncated SVD model also resulted in an RMSE of approximately 0.9996, similar to the basic SVD model. The precision, recall, and F1-score were again all 0.0, indicating that this model did not perform well in our recommendation context. The linear assumption of latent factors likely affected its performance.

3. Alternating Least Squares (ALS)

	•	Root Mean Squared Error (RMSE): 0.0199
    
	•	Precision: 1.0
    
	•	Recall: 0.0
    
	•	F1-Score: 0.0
    

Analysis:
The ALS model achieved a significantly lower RMSE of 0.0199, suggesting better predictive accuracy compared to the SVD models. However, the precision was 1.0 while the recall and F1-score were 0.0, indicating that the model identified very few items as relevant (leading to precision) but missed many actual relevant items (resulting in zero recall). This suggests potential underfitting, likely due to the high sparsity of the user-item matrix. Further hyperparameter tuning is needed to improve the model’s performance.

4. Deep Learning Model

	•	Root Mean Squared Error (RMSE): 0.0183
    
	•	Precision: 1.0
    
	•	Recall: 1.0
    
	•	F1-Score: 1.0
    

Analysis:
The deep learning model achieved the lowest RMSE of 0.0183, indicating excellent predictive accuracy. However, the precision, recall, and F1-score were all 1.0, which is typically unrealistic and suggests that the model is overfitting to the training data. This overfitting means that the model may not generalize well to new, unseen data.

Conclusion


	•	SVD and Truncated SVD: Both SVD models failed to perform well due to their assumption of linear latent factors, resulting in high RMSE and zero precision, recall, and F1-score.
    
	•	ALS: The ALS model showed promising RMSE results, but the issue of underfitting in the top-k recommendations highlights the need for further tuning and addressing the sparsity of the data.
    
	•	Deep Learning: The deep learning model achieved the best RMSE, but its perfect scores in precision, recall, and F1-score indicate severe overfitting, which limits its practicality.
    

Recommendations


	•	Hyperparameter Tuning: Further tuning of the ALS model is essential to improve its performance and mitigate the underfitting issue.
    
	•	Overfitting Mitigation: For the deep learning model, applying regularization techniques, dropout, and cross-validation will help reduce overfitting and improve generalization to new data.
    
	•	Hybrid Approaches: Exploring hybrid recommendation systems that combine collaborative filtering with content-based methods can leverage additional user and item metadata, potentially improving overall recommendation quality.
    

By iterating on these models and incorporating the suggested improvements, we can develop a more accurate and reliable cross-platform recommendation system for books, music, and movies.